Scrapping from Wikipedia page that contains the neighborhoods in Toronto and transforming the data into a pandas dataframe using BeautifulSoup

=>Imporingt required libraries & scrape the Wikipedia page

In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [15]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [16]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

=>Creating and cleaning the dataframe

In [17]:
df = pd.DataFrame(row)
x = df[0].str.split('\n', expand=True)
y = x.rename(columns=x.iloc[0])
z = y.drop(y.index[0])
z.head()

,,Postal Code,,Borough,,Neighbourhood,
1,,M1A,,Not assigned,,Not assigned,
2,,M2A,,Not assigned,,Not assigned,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


=>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [18]:
toronto = z[z.Borough != 'Not assigned']
toronto.head()

,,Postal Code,,Borough,,Neighbourhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


=>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [21]:
combined = toronto.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
combined.reset_index(inplace = True)
combined.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


=>If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [26]:
def change_value(a):    
    return ', '.join(sorted(a['Neighbourhood'].tolist()))
                    
df_2 = combined.groupby(['Postal Code', 'Borough'])
final_df = df_2.apply(change_value).reset_index(name='Neighbourhood')

In [27]:
final_df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


=>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [28]:
final_df.shape

(103, 3)